In [ ]:
!pip install ../input/torchlrfinder/torch_lr_finder-0.2.1-py3-none-any.whl
!pip install ../input/torchsummary/torchsummary-1.5.1-py3-none-any.whl

***
### notes

- v2: Reduces the number of channels to 32 & 64, adds an intermediate output layer.
- v3: Reduces `weight_decay` to `1e-4`.
- v4: IterativeImputer set to 1 iteration & 50 features.
- v5: IterativeImputer set to 1 iteration & 30 features. `CNN1D(*, sign_size=22, cha_1=16, cha_2=32, cha_3=32, dropout_in1=0.2, dropout_in2=0.1, dropout_mid=0.3, dropout_out=0.2)`.
- v6: Hyperparams: `{'sign_size': 22, 'cha_1': 16, 'cha_2': 32, 'cha_3': 32, 'input_dim': 131, 'output_dim': 5, 'dropout_in1': 0.25, 'dropout_in2': 0.4, 'dropout_mid': 0.4, 'dropout_out': 0.25, 'weight_decay': 0.0001, 'pct_start': 0.1, 'max_lr': 0.07}`.
- v7: Keeps state of optimizer, hyperparams: `sign_size=16, cha_1=32, cha_2=64, cha_3=64, dropout_in1=0.25, dropout_in2=0.3, dropout_mid=0.4, dropout_out=0.2`.
- v8: Removed intermediate output layer, input activation set to `celu`, hyperparams: `sign_size=8, cha_1=32, cha_2=64, cha_3=64, dropout_in1=0.25, dropout_in2=0.3, dropout_mid=0.3, dropout_out=0.15)`.
- v9: Add NormalLinear layer
- v10: Major refactoring of model structure (depthwise convolution, higher regularization, lower lr, swa).
- v11: Hyperparams: `sign_size=16, cha_input=32, cha_hidden=48, K=2, dropout_input=0.2, dropout_hidden=0.25, dropout_output=0.25, weight_decay=1e-4, max_lr=1e-2, pct_start=0.1`.
- v11: Trained with lower regularization dropout_input=0.2, dropout_hidden=0.25, dropout_output=0.15 and momentum=0.8 on SWA.
- v12: Changed hyperparams to `sign_size=8, cha_input=32, cha_hidden=64, K=2, dropout_input=0.20, dropout_hidden=0.25, dropout_output=0.15, weight_decay=1e-3`.
- v13: Changed hyperparams `dropout_input=0.25, dropout_hidden=0.3, dropout_output=0.15`. SWA has st (15 epochs) and lt (30 epochs) version.

In [ ]:
import copy
import numpy as np 
import pandas as pd
from pathlib import Path
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from collections import OrderedDict

from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import BayesianRidge
from category_encoders.one_hot import OneHotEncoder

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset,TensorDataset,DataLoader
from torch_lr_finder import LRFinder
from torchsummary import summary

# custom modules
import sys
sys.path.append("../usr/lib/janestreet_torch_utils")
from janestreet_torch_utils import Monitor, train_step, valid_step

def set_seed(seed):
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    
set_seed(2)

torch.backends.cudnn.enabled = True
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = True
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
def utility_score(date, weight, resp, action):
    """
    Fast computation of utility score
    """
    date = date.astype(int)
    count_i = len(np.unique(date))
    Pi = np.bincount(date, weight * resp * action)
    t = np.sum(Pi) / np.sqrt(np.sum(Pi ** 2)) * np.sqrt(250 / count_i)
    u = np.clip(t, 0, 6) * np.sum(Pi)
    return -u

In [ ]:
def cat_encoder(X):
    """
    Fast one-hot encoding of feature_0
    """
    X["feature_00"] = 0
    idx00 = X.query("feature_0 == -1").index
    X.loc[idx00,"feature_00"] = 1
    
    X["feature_01"] = 0
    idx01 = X.query("feature_0 == 1").index
    X.loc[idx01,"feature_01"] = 1
    
    return X.iloc[:,1:]

In [ ]:
def show_metrics(monitor):
    x = np.arange(len(monitor.train_loss))
    
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(21, 7))
    
    ax1 = axes[0]
    ax2 = ax1.twinx()
    ax1.plot(x, monitor.train_loss, 'go-', label="train_loss")
    ax2.plot(x, monitor.train_metric, 'ro-', label="train_metric")
    plt.legend(loc="best")
    ax1.set_xlabel('epochs')
    ax1.set_ylabel('loss')
    ax1.set_title("Training")
    plt.grid()
    
    ax1 = axes[1]
    ax2 = ax1.twinx()
    ax1.plot(x, monitor.valid_loss, 'go-', label="valid_loss")
    ax2.plot(x, monitor.valid_metric, 'ro-', label="valid_metric")
    plt.legend(loc="best")
    ax1.set_xlabel('epochs')
    ax2.set_ylabel('metric')
    ax1.set_title("Validation")
    plt.grid()
    
    plt.show()

***
### preparing the data

In [ ]:
root = Path("../input/janestreet-preprocessing")

train = pd.read_parquet(root/"train.parquet")
features = pd.read_parquet(root/"features.parquet")

train.info()

In [ ]:
train = train.query("date > 85").query("weight > 0").reset_index(drop=True)

input_features = [col for col in train.columns if "feature" in col]
resp_cols = ['resp', 'resp_1', 'resp_2', 'resp_3', 'resp_4']
w_cols = ["w", "w1", "w2", "w3", "w4"]

X_dset = train.loc[:,input_features].copy()
y_dset = (train.loc[:,resp_cols] > 0).astype(int).copy()
w_dset = train.loc[:, w_cols].copy()
dwr_dset = train.loc[:, ["date","weight","resp"]].copy()

In [ ]:
%%time

with open("../input/janestreet-imputer/imputer_f0m1.pickle", "rb") as file:
    imputer_f0m1 = pickle.load(file)
    file.close()
    
with open("../input/janestreet-imputer/imputer_f0p1.pickle", "rb") as file:
    imputer_f0p1 = pickle.load(file)
    file.close()
    
    
idx_f0m1 = X_dset.query("feature_0 == -1").index
X_dset.loc[idx_f0m1, input_features[1:]] = imputer_f0m1.transform(X_dset.loc[idx_f0m1, input_features[1:]])

idx_f0p1 = X_dset.query("feature_0 ==  1").index
X_dset.loc[idx_f0p1, input_features[1:]] = imputer_f0p1.transform(X_dset.loc[idx_f0p1, input_features[1:]])

In [ ]:
X_dset = cat_encoder(X_dset)
input_features = X_dset.columns.tolist()

***
### model definition

In [ ]:
class NormalLinear(nn.Module):
    """ 
    Linear layer with normalized weights
    """
    def __init__(self, size_in, size_out, bias=True):
        super(NormalLinear, self).__init__()
        self.size_in, self.size_out = size_in, size_out
        # weights vector
        weights_v = torch.Tensor(size_out, size_in)
        nn.init.kaiming_uniform_(weights_v, a=np.sqrt(5)) 
        self.weights_v = nn.Parameter(weights_v)
        # weights magnitude
        weights_m = torch.norm(weights_v, dim=1, keepdim=True)
        self.weights_m = nn.Parameter(weights_m.clone().detach())
        
        if bias:
            bias_v = torch.Tensor(size_out)    
            fan_in, _ = nn.init._calculate_fan_in_and_fan_out(weights_v)
            bound = 1 / np.sqrt(fan_in)
            nn.init.uniform_(bias_v, -bound, bound)
            self.bias = nn.Parameter(bias_v)
        else:
            self.register_parameter('bias', None)
            
    def _compute_weights(self):
        norm_per_output = torch.norm(self.weights_v, dim=1, keepdim=True)
        return self.weights_m * torch.div(self.weights_v, norm_per_output)
            
    def forward(self, x):
        weights = self._compute_weights()
        return nn.functional.linear(x, weights, self.bias)


class CNN1D(nn.Module):
    def __init__(self, input_dim, output_dim, sign_size=16, cha_input=32, cha_hidden=32, K=2,
                 dropout_input=0.2, dropout_hidden=0.2, dropout_output=0.2):
        super(CNN1D, self).__init__()

        hidden_size = sign_size*cha_input
        sign_size1 = sign_size
        sign_size2 = sign_size//2
        output_size = (sign_size//4) * cha_hidden

        self.hidden_size = hidden_size
        self.cha_input = cha_input
        self.cha_hidden = cha_hidden
        self.K = K
        self.sign_size1 = sign_size1
        self.sign_size2 = sign_size2
        self.output_size = output_size
        self.dropout_input = dropout_input
        self.dropout_hidden = dropout_hidden
        self.dropout_output = dropout_output

        self.batch_norm1 = nn.BatchNorm1d(input_dim)
        self.dropout1 = nn.Dropout(dropout_input)
        self.dense1 = NormalLinear(input_dim, hidden_size, bias=False)

        # 1st conv layer
        self.batch_norm_c1 = nn.BatchNorm1d(cha_input)
        self.conv1 = nn.Conv1d(cha_input, cha_input*K, kernel_size = 5, stride = 1, padding=2,  groups=cha_input, bias=False)

        self.ave_po_c1 = nn.AdaptiveAvgPool1d(output_size = sign_size2)

        # 2nd conv layer
        self.batch_norm_c2 = nn.BatchNorm1d(cha_input*K)
        self.dropout_c2 = nn.Dropout(dropout_hidden)
        self.conv2 = nn.Conv1d(cha_input*K, cha_hidden, kernel_size = 3, stride = 1, padding=1, bias=False)

        # 3rd conv layer
        self.batch_norm_c3 = nn.BatchNorm1d(cha_hidden)
        self.dropout_c3 = nn.Dropout(dropout_hidden)
        self.conv3 = nn.Conv1d(cha_hidden, cha_hidden, kernel_size = 3, stride = 1, padding=1, bias=False)

        # 4th conv layer
        self.batch_norm_c4 = nn.BatchNorm1d(cha_hidden)
        self.conv4 = nn.Conv1d(cha_hidden, cha_hidden, kernel_size = 5, stride = 1, padding=2, groups=cha_hidden, bias=False)

        self.avg_po_c4 = nn.AvgPool1d(kernel_size=4, stride=2, padding=1)

        self.flt = nn.Flatten()

        self.batch_norm2 = nn.BatchNorm1d(output_size)
        self.dropout2 = nn.Dropout(dropout_output)
        self.dense2 = NormalLinear(output_size, output_dim, bias=False)

    def forward(self, x):

        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.celu(self.dense1(x))

        x = x.reshape(x.shape[0], self.cha_input, self.sign_size1)

        x = self.batch_norm_c1(x)
        x = F.relu(self.conv1(x))

        x = self.ave_po_c1(x)

        x = self.batch_norm_c2(x)
        x = self.dropout_c2(x)
        x = F.relu(self.conv2(x))
        x_s = x

        x = self.batch_norm_c3(x)
        x = self.dropout_c3(x)
        x = F.relu(self.conv3(x))

        x = self.batch_norm_c4(x)
        x = self.conv4(x)
        x =  x + x_s
        x = F.relu(x)

        x = self.avg_po_c4(x)

        x = self.flt(x)

        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = self.dense2(x)

        return x

In [ ]:
class BCELabelSmoothing(nn.Module):
    def __init__(self, label_smoothing=0.0):
        super(BCELabelSmoothing, self).__init__()
        self.label_smoothing = label_smoothing
        self.bce_loss = torch.nn.functional.binary_cross_entropy_with_logits
        
    def forward(self, prediction, target, weight=None):
        target_smooth = target*(1.0 - self.label_smoothing) + 0.5*self.label_smoothing
        if weight is None:
            loss = self.bce_loss(prediction, target_smooth, reduction="mean")
        else:
            loss = self.bce_loss(prediction, target_smooth, weight, reduction="sum") / torch.sum(weight)
        return loss

bce_loss = BCELabelSmoothing(label_smoothing=1e-2)

***
### model training: 1st step

In [ ]:
train_idx = train.query("date < 450").index
valid_idx = train.query("date >= 450").index

train_dset = TensorDataset(torch.tensor(X_dset.loc[train_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[train_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[train_idx].values, dtype=torch.float),
                          )

valid_dset = TensorDataset(torch.tensor(X_dset.loc[valid_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[valid_idx].values, dtype=torch.float),
                          )

dataset_sizes = {'train': len(train_dset), 'valid': len(valid_dset)}
train_dataloader = DataLoader(train_dset, batch_size=2048, shuffle=True, num_workers=2, pin_memory=True, drop_last=True)
valid_dataloader = DataLoader(valid_dset, batch_size=len(valid_dset), shuffle=False, num_workers=2, pin_memory=True, drop_last=True)

print("Number of step per epoch:", len(train_dset)//2048)

In [ ]:
_model = CNN1D(input_dim=len(input_features), output_dim=len(resp_cols),
               sign_size=8, cha_input=32, cha_hidden=64, K=2,
               dropout_input=0.25, dropout_hidden=0.3, dropout_output=0.15)
_model = _model.to(device)

_optimizer = torch.optim.SGD(_model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-3)
lr_finder = LRFinder(_model, _optimizer, bce_loss, device="cuda")
lr_finder.range_test(train_dataloader, start_lr=1e-4, end_lr=1e1, num_iter=652*2, step_mode="exp")
lr_finder.plot(show_lr=1e-2)
plt.show()

_optimizer = torch.optim.SGD(_model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-3)
lr_finder = LRFinder(_model, _optimizer, bce_loss, device="cuda")
lr_finder.range_test(train_dataloader, start_lr=1e-4, end_lr=1e1, num_iter=652*2, step_mode="exp")
lr_finder.plot(show_lr=1e-2)
plt.show()

In [ ]:
model = CNN1D(input_dim=len(input_features), output_dim=len(resp_cols),
              sign_size=8, cha_input=32, cha_hidden=64, K=2,
              dropout_input=0.25, dropout_hidden=0.3, dropout_output=0.15)
model = model.to(device)
summary(model, input_size=(len(input_features),))

In [ ]:
models_history = list()

optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=1e-3)
scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer, 
    max_lr=1e-2,
    epochs=50,
    pct_start=0.3, 
    anneal_strategy='cos', 
    cycle_momentum=True, 
    base_momentum=0.8, 
    max_momentum=0.9, 
    div_factor=1e1,
    final_div_factor=1e0,
    steps_per_epoch=len(train_dataloader),
    verbose=False)

monitor = Monitor(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    patience=10,
    metric_fn=utility_score,
    experiment_name=f'cnn1d',
    num_epochs=50,
    dataset_sizes=dataset_sizes,
    early_stop_on_metric=False,
    lower_is_better=True)

for epoch in monitor.iter_epochs:
    train_step(model, train_dataloader, optimizer, monitor, bce_loss, scheduler=scheduler, clip_value=None)    
    early_stop = valid_step(model, valid_dataloader, optimizer, monitor, bce_loss)
    models_history.append(copy.deepcopy(model))
    if early_stop: break

In [ ]:
show_metrics(monitor)

In [ ]:
# saves all the models
for epoch,_model in enumerate(models_history):
    torch.save(_model.state_dict(), f"./cnn1d-step1-epoch{epoch+1}.pt")

In [ ]:
# SWA around optimal on validation data
swa_model = torch.optim.swa_utils.AveragedModel(model)
best_epoch = len(models_history)-11

for _model in models_history[best_epoch:]:
    swa_model.update_parameters(_model)
    
torch.optim.swa_utils.update_bn(train_dataloader, swa_model, device=device)

In [ ]:
# puts state_dict of SWA in the format of original model
state_dict = OrderedDict()
state_dict_swa = swa_model.state_dict()

for key,params in state_dict_swa.items():
    if key == "n_averaged": continue
    key = key.replace("module.","")
    state_dict[key] = params
    
# restore model to best averaged state
model.load_state_dict(state_dict)

***
### model training: 2nd step

In [ ]:
train_idx = train.query("date < 450").index
valid_idx = train.query("date >= 450").index

train_dset = TensorDataset(torch.tensor(X_dset.loc[:].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[:].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[:].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[:].values, dtype=torch.float),
                          )

valid_dset = TensorDataset(torch.tensor(X_dset.loc[valid_idx].values, dtype=torch.float), 
                           torch.tensor(y_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(w_dset.loc[valid_idx].values, dtype=torch.float),
                           torch.tensor(dwr_dset.loc[valid_idx].values, dtype=torch.float),
                          )

dataset_sizes = {'train': len(train_dset), 'valid': len(valid_dset)}
train_dataloader = DataLoader(train_dset, batch_size=2048, shuffle=True, num_workers=2)
valid_dataloader = DataLoader(valid_dset, batch_size=len(valid_dset), shuffle=False, num_workers=2)

print("Number of step per epoch:", len(train_dset)//2048)

In [ ]:
_optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.8, weight_decay=1e-3)
lr_finder = LRFinder(model, _optimizer, bce_loss, device="cuda")
lr_finder.range_test(train_dataloader, start_lr=1e-4, end_lr=1e1, num_iter=652*4, step_mode="exp")
lr_finder.plot(show_lr=1e-2)
plt.show()

In [ ]:
# restore to state of best epoch
model.load_state_dict(state_dict)
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3, momentum=0.8, weight_decay=1e-3)

In [ ]:
scheduler = torch.optim.swa_utils.SWALR(optimizer, swa_lr=5e-3, anneal_strategy="cos", anneal_epochs=5)

models_history = list()

monitor = Monitor(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    patience=35,
    metric_fn=utility_score,
    experiment_name=f'cnn1d',
    num_epochs=35,
    dataset_sizes=dataset_sizes,
    early_stop_on_metric=False,
    lower_is_better=True)

for epoch in monitor.iter_epochs:
    train_step(model, train_dataloader, optimizer, monitor, bce_loss, scheduler=None, clip_value=None)    
    valid_step(model, valid_dataloader, optimizer, monitor, bce_loss)
    scheduler.step()
    
    models_history.append(copy.deepcopy(model))
    
    # save models along the path of swa
    torch.save(model.state_dict(), f"./cnn1d-epoch{epoch+1}.pt")

In [ ]:
show_metrics(monitor)

In [ ]:
# SWA short training
start_epoch = 5
end_epoch = 20

swa_model_st = torch.optim.swa_utils.AveragedModel(model)
swa_model_st = swa_model_st.to(device)

for _model in models_history[start_epoch:end_epoch]:
    swa_model_st.update_parameters(_model)
    
torch.optim.swa_utils.update_bn(train_dataloader, swa_model_st, device=device)

# save final model for inference
torch.save(swa_model_st.state_dict(), "./cnn1d-swa-st.pt")

In [ ]:
# SWA long training
start_epoch = 5
end_epoch = 35

swa_model_lt = torch.optim.swa_utils.AveragedModel(model)
swa_model_lt = swa_model_lt.to(device)

for _model in models_history[start_epoch:end_epoch]:
    swa_model_lt.update_parameters(_model)
    
torch.optim.swa_utils.update_bn(train_dataloader, swa_model_lt, device=device)

# save final model for inference
torch.save(swa_model_lt.state_dict(), "./cnn1d-swa-lt.pt")

***